# Problem 1 #

Using the data from table 2-3 from D&H page 64, compute six group and total delayed neutron fraction for $^{235}$U thermal fission.

In [1]:
import numpy as np
import pandas as pd

In [2]:
delayed_neutrons_per_thermal_fission = 0.01668
neutrons_per_thermal_fission = 2.43
delayed_group = pd.DataFrame(
    {
        "Group": range(1,7),
        "Half Life": [54.51, 21.84, 6.00, 2.23, 0.496, 0.179],
        "Relative Yield": [0.038, 0.213, 0.188, 0.407, 0.128, 0.026]
    }
)
delayed_group.set_index("Group", inplace=True)
delayed_group

,Half Life,Relative Yield
Group,,
1,54.510,0.038
2,21.840,0.213
3,6.000,0.188
4,2.230,0.407
5,0.496,0.128
6,0.179,0.026


In [3]:
delayed_group["Delayed Neutron Fraction"] =\
    delayed_group["Relative Yield"] * delayed_neutrons_per_thermal_fission / neutrons_per_thermal_fission
delayed_group

,Half Life,Relative Yield,Delayed Neutron Fraction
Group,,,
1,54.510,0.038,0.000261
2,21.840,0.213,0.001462
3,6.000,0.188,0.001290
4,2.230,0.407,0.002794
5,0.496,0.128,0.000879
6,0.179,0.026,0.000178


In [4]:
print "Total delayed neutron fraction: {0}".format(sum(delayed_group["Delayed Neutron Fraction"]))

Total delayed neutron fraction: 0.00686419753086


In [5]:
# verification
(0.01668) / (2.43 + 0.01668)

0.006817401540046103

# Problem 2 #

Repeate the solution of the point reactor kinetics equation with one effective delayed group for a constant reactivity addition $\rho_0$ using the Laplace transform method.

# Problem 3 #

A PWR filled with $^{235}$U is initially operating at a power level of 1000 MWt. What is the power of the reactor after 60 seconds for reactivity insertions of -20 cents and +20 cents?

Assume a prompt neutron lifetime of 0.1 milliseconds

In [17]:
tau_prompt = 1e-4

In [24]:
delayed_group["Decay Constant"] = np.log(2.) / delayed_group["Half Life"]
delayed_group.ix[:, delayed_group.columns.difference(["Relative Yield"])]

,Decay Constant,Delayed Neutron Fraction,Half Life
Group,,,
1,0.012716,0.000261,54.510
2,0.031738,0.001462,21.840
3,0.115525,0.001290,6.000
4,0.310828,0.002794,2.230
5,1.397474,0.000879,0.496
6,3.872331,0.000178,0.179


In [19]:
beta = sum(delayed_group["Delayed Neutron Fraction"])

In [20]:
theta = delayed_group["Delayed Neutron Fraction"].dot(delayed_group["Half Life"]) + (1. - beta) * tau_prompt
theta

0.060689942123456783

In [29]:
print 'beta: {0}'.format(beta)
print "theta: {0}".format(theta)

beta: 0.00686419753086
theta: 0.0606899421235


In [10]:
def k(reactivity):
    return 1. / (1. - reactivity)

def tau(theta, k):
    return theta / np.log(k)

In [11]:
def power(k, t, initial=1000):
    period = tau(theta, k)
    p = initial * np.exp(t / period)
    
    return p

In [30]:
power_plus_twenty_cents = power(k(0.2 * beta), 60)
power_plus_twenty_cents

3889.0508282555074

In [31]:
power_minus_twenty_cents = power(k(-0.2 * beta), 60)
power_minus_twenty_cents

257.61170155452254

# Problem 4 #

Using the data in Table 2-3 from D&H page 64 and Figure 2-20 from D&H page 61 for $^{235}$U solve the inhour equation below to obtain the decay constants, $s$, for reactivity values of \$-0.20 and \$+0.20.

\begin{equation}
    \rho = \Lambda s + \sum_{i=1}^6 \frac{\beta_i s}{s + \lambda_i}
\end{equation}

Using the reactivity function defined in number 3 and $\Lambda = \frac{1}{k}$

The roots of this equation come when $s + \lambda_i = 0 \rightarrow s = -\lambda_i$